In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Read datasets to pandas dataframe
df_train = pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/train.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/test.csv')
df_sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/sample_submission.csv')

# Missing values?

In [ ]:
 df_train.isna().any().sum()

In [ ]:
df_test.isna().any().sum()

# Ok, then...

In [ ]:
df_train.describe()

In [ ]:
df_train.groupby('Cover_Type').size()

In [ ]:
df_train = df_train[df_train['Cover_Type']!=5]

# Data Preprocessing

In [ ]:
train=df_train
test=df_test

In [ ]:
X = train.iloc[:, 1:-1].copy()
# Get the target
y = train.Cover_Type.copy()

# Create test X, drop ids.
test_X = test.iloc[:, 1:].copy()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validate, Y_train, Y_validate = train_test_split( X, y, test_size=0.2, random_state=2)
print ('Train set:', X_train.shape,  Y_train.shape)
print ('Validation set:', X_validate.shape,  Y_validate.shape)

In [ ]:
X_train

# LGBM

In [ ]:
# import optuna

In [ ]:
# def objective(trial):
    
#     train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.20, stratify = y)
#     dtrain = lgb.Dataset(train_x, label=train_y)
#     dtest = lgb.Dataset(valid_x, label=valid_y)
#     param = {
#         "is_unbalance": True,
#         'objective': 'multiclass',
#         "num_class": 8, #df_train['Cover_Type'].nunique()+1
#         'metric': "multi_logloss",
#         'verbosity': -1,
#         "num_threads": -1,
#         'num_iterations': 1000,
#         'n_estimators': 7000
#      }
    
#     pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "multi_logloss")
    
#     gbm = lgb.train(param, dtrain, valid_sets= [dtest])
#     y_pred = gbm.predict(valid_x)
#     test_preds = [np.argmax(x) for x in y_pred]
#     accuracy = accuracy_score(valid_y, test_preds)
#     return accuracy

In [ ]:
# import lightgbm as lgb
# from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.20, stratify = y)
# dtrain = lgb.Dataset(train_x, label=train_y)
# dtest = lgb.Dataset(valid_x, label=valid_y)

# params = {
#     'learning_rate': 0.05992665960877217,
#     'boosting': 'gbdt',
#     'lambda_l2': 2.537222734343992, 
#     'num_leaves': 225, 'max_depth': 10,
#     "is_unbalance" : True,
#     'objective': 'multiclass',
#     "num_class": 8, #df_train['Cover_Type'].nunique()+1
#     'metric': "multi_logloss",
#     'verbosity': -1,
#     "num_threads": -1,
#     'num_iterations': 50
#     }

# gbm = lgb.train(params, dtrain, valid_sets = [dtest])
# y_pred = gbm.predict(valid_x)
# test_preds = [np.argmax(x) for x in y_pred]
# print(accuracy_score(valid_y, test_preds))

In [ ]:
# pred = gbm.predict(df_test)
# pred = [np.argmax(x) for x in pred]
# df_subb = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv')
# df_subb['Cover_Type'] = pred
# df_subb = df_subb.set_index('Id')
# df_subb

In [ ]:
# df_subb.to_csv('lgb_subb.csv')

In [ ]:
from lightgbm import LGBMClassifier

lgb_params = {
    'objective' : 'multiclass',
    'metric' : 'multi_logloss',
    'device' : 'gpu',
}

lgbmmodel = LGBMClassifier(**lgb_params) 

lgbmmodel.fit(X_train,Y_train,
               early_stopping_rounds=200,
               eval_set=[(X_validate,Y_validate)],
               verbose=True)

# R^2 for training data
lgbmmodel.score(X_train,Y_train)

In [ ]:
# View sample submission
df_sample_submission

In [ ]:
df_lgbm_submission = df_sample_submission
df_lgbm_submission['Cover_Type'] = lgbmmodel.predict(test_X).astype('int')
df_lgbm_submission.to_csv("submission.csv",index=False)